In [1]:
import os
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
from bokeh.embed import components
from bokeh.models import Range1d
from bokeh.models import NumeralTickFormatter


import numpy as np

In [2]:
import pandas as pd
import math
import statsmodels.api as sm


In [3]:
output_notebook()
def make_cfr():
    df = pd.read_csv('/home/henry/Downloads/results-20201226-192801.csv')
    df['date'] = pd.to_datetime(df['date'])
    #df = df[df['date'] > datetime.datetime(2020, 5, 5)]
    df = df[df['cfr'] < .06]
    p = figure(x_axis_type = 'datetime')
    dates = df['date'].tolist()
    df2 = df[df['date'] > datetime.datetime(2020,8,1)]
    print(np.mean(df2['cfr'])/.006)
    cfr = df['cfr']
    p.line(x = dates, y = cfr)
    return p
    
show(make_cfr())


Loading BokehJS ...

2.7276757779607217


In [4]:
def make_true_infections(window = 7):
    df = pd.read_csv('/home/henry/Downloads/results-20201226-203941.csv')
    df['date'] = pd.to_datetime(df['date'])
    df_ny = df[df['state'] == 'New York']
    df_ny = df_ny.sort_values(by = 'date')
    df_wa = df[df['state'] == 'Washington']
    df_wa = df_wa.sort_values(by = 'date')
    df_ca = df[df['state'] == 'California']
    df_ca = df_ca.sort_values(by = 'date')
    df_fl = df[df['state'] == 'Florida']
    df_fl = df_fl.sort_values(by = 'date')
    df_ri = df[df['state'] == 'Rhode Island']
    df_ri = df_ri.sort_values(by = 'date')
    df_nb = df[df['state'] == 'Nebraska']
    df_nb = df_nb.sort_values(by = 'date')


    p = figure(x_axis_type = 'datetime')
    dates = df_ny['date'].tolist()
    #p.line(x = dates, y = df_ny['infections_per_100k'].rolling(window).mean(), 
           #legend_label = 'New York')
    p.line(x = df_wa['date'], y = df_wa['infections_per_100k'].rolling(window).mean(), 
           legend_label = 'WA', color = 'red')
    p.line(x = df_ca['date'], y = df_ca['infections_per_100k'].rolling(window).mean(), 
           legend_label = 'CA', color = 'green')
    p.line(x = df_fl['date'], y = df_fl['infections_per_100k'].rolling(window).mean(), 
           legend_label = 'Fl', color = 'black')
    p.line(x = df_ri['date'], y = df_ri['infections_per_100k'].rolling(window).mean(), 
           legend_label = 'RI', color = 'yellow')
    p.line(x = df_nb['date'], y = df_nb['infections_per_100k'].rolling(window).mean(), 
           legend_label = 'NB', color = 'orange')


    p.legend.location = 'top_left'


    return p

show(make_true_infections())


    

In [5]:
def pos_wa(window = 7, plot_height = 350, plot_width = 350):
    df = pd.read_csv('/home/henry/Downloads/results-20201229-005934.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df.assign(pos_7 = df['positive_increase'].rolling(window).mean())
    df = df.assign(tests_7 = df['total_test_results_increase'].rolling(window).mean())
    df = df.assign(pos_per_7 = df['pos_7']/df['tests_7'] * 100)
    df = df[df['pos_per_7'] < 10]
    df = df[df['date'] >= datetime.datetime(2020,2,12)]
    dates = df['date'].tolist()
    days_since_feb_12 = []
    for i in dates:
        diff = i - datetime.datetime(2020,2, 12)
        days_since_feb_12.append((i, diff.days))
    df = df.assign(days_since_feb_12 = [x[1] for x in days_since_feb_12])
    df = df.assign(prevelance = 1500/(df['days_since_feb_12'] + 50) * np.sqrt(df['pos_per_7']/100) + 2)
    df = df.assign(cases_t = df['cases'] * df['prevelance'])
    df = df.assign(ifr = df['deaths']/df['cases_t'])
    df = df[(df['ifr'] < .03) & (df['ifr'] >= 0)]
    #df = df.assign(cases_t2 = df['deaths']/(df['days_since_feb_12'] * 0.009039 - 1.9869 ))
    df = df.assign(cases_t2 = df['deaths']/.006)

    x = df['days_since_feb_12'].tolist()
    X = list(zip(*[x]))
    y = df['ifr'].tolist()
    xm = sm.add_constant(X)
    model = sm.OLS(y, xm) 
    result = model.fit()

    y_hat = result.predict(xm)

    print(result.params[1], result.params[0])

    p = figure(x_axis_type= 'datetime', plot_height = plot_height, 
              plot_width = plot_width, title = 'Positivity Rate')
    p.line(x = df['date'], y = df['pos_per_7'])
    p2 = figure(x_axis_type = 'datetime', plot_height = plot_height,
               plot_width = plot_width, title = 'Cases')
    p2.line(x = df['date'], y = df['cases_t'])
    p3 = figure(plot_height = plot_height, plot_width = plot_width, title = 'IFR')
    p3.circle(x= x, y = y)
    p3.line(x = x, y = y_hat)
    p4 = figure(x_axis_type = 'datetime', plot_width = plot_width, plot_height = plot_height,
               title = 'cases')
    p4.line(x = df['date'], y = df['cases_t2'])
    print(np.mean(df['ifr']))
    grid = gridplot([p, p2,p3, p4], ncols = 2)
    return grid


show(pos_wa())

-1.9869574572441047e-05 0.009039630024386307
0.005495685543438674


In [17]:
def days_since_feb12(dates):
    days_since_feb_12 = []
    for i in dates:
        diff = i - datetime.datetime(2020,2, 12)
        days_since_feb_12.append(diff.days)
    return days_since_feb_12

In [99]:
def get_df_adj(window = 7):
    df = pd.read_csv('/home/henry/Downloads/results-20201231-212430.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['state'] == 'Washington']
    df = df.assign(pos_7 = df['positive'].rolling(window).mean())
    df = df.assign(num_tests_7 = df['num_tests'].rolling(window).mean())
    #df = df.assign(pos_per = df['positive']/df['num_tests'])
    df = df.assign(pos_per = df['pos_7']/df['num_tests_7'])
    df = df.assign(deaths_7 = df['deaths'].rolling(window).mean())
    df = df.assign(three_weeks = df['date'] - datetime.timedelta(days = 21))


    df = df.assign(days_since_feb_12 =  days_since_feb12(dates = df['date'].tolist()))
    df = df[(df['days_since_feb_12'] >= 0) & (df['pos_per'] != math.inf)
            & (df['pos_per'] is not math.nan) & (df['num_tests'] > 0)
           ]
    df = df.assign(prevelance = 1500/(df['days_since_feb_12'] + 50) * np.sqrt(df['pos_per'])/100 + 2)
    df = df.assign(infections = df['cases'] * df['prevelance'])
    df = df.assign(infections_by_ifr = df['deaths_7'] / (0.00903  -1.9869e-05 * df['days_since_feb_12']))


    return df

get_df_adj()

    

,state,date,positive,num_tests,cases,deaths,pos_7,num_tests_7,pos_per,deaths_7,three_weeks,days_since_feb_12,prevelance,infections,infections_by_ifr
77,Washington,2020-02-29,3,42,3,1,1.571429,6.000000,0.261905,0.142857,2020-02-08,17,2.114575,6.343724,16.435045
80,Washington,2020-03-01,8,75,7,2,2.571429,16.714286,0.153846,0.428571,2020-02-09,18,2.086522,14.605653,49.418097
85,Washington,2020-03-02,17,188,6,3,4.857143,43.571429,0.111475,0.857143,2020-02-10,19,2.072582,12.435495,99.063155
93,Washington,2020-03-03,16,226,9,4,7.000000,75.857143,0.092279,1.428571,2020-02-11,20,2.065094,18.585850,165.485267
106,Washington,2020-03-04,34,251,15,1,11.571429,111.714286,0.103581,1.571429,2020-02-12,21,2.067994,31.019913,182.453733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14980,Washington,2020-12-24,2315,30191,2437,23,2276.714286,26822.571429,0.084881,14.000000,2020-12-03,316,2.011940,4903.098444,5088.326072
15031,Washington,2020-12-25,2891,28560,420,0,2269.714286,30902.571429,0.073447,14.285714,2020-12-04,317,2.011077,844.652247,5229.937059
15133,Washington,2020-12-27,3626,61569,273,0,2017.000000,29600.285714,0.068141,16.142857,2020-12-06,319,2.010611,548.896894,5997.073746
15235,Washington,2020-12-29,1953,57883,2145,133,1719.571429,29852.428571,0.057602,31.285714,2020-12-08,321,2.009704,4310.814449,11796.799641


In [52]:
def get_standard():
    df = pd.read_csv('data/latest_all_estimates_states.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['state'] == 'WA']
    #df = df[df['date'] == datetime.datetime(2020, 8, 1)]
    cases = []
    b = 0
    l =df['total_infected_mean'].tolist()
    for counter, i in enumerate(l):
        if counter == 0:
            cases.append(i)
        else:
            cases.append(i - l[counter - 1])
    df = df.assign(cases = cases)
    return df
get_standard()

,date,state,total_cases,total_deaths,total_tests,current_hospitalized,daily_positive_7day_ma,daily_deaths_7day_ma,daily_tests_7day_ma,positivity_rate_orig_7day_ma,...,perc_current_infected_mean,perc_current_infected_lower,perc_current_infected_upper,total_infected_mean,total_infected_lower,total_infected_upper,perc_total_infected_mean,perc_total_infected_lower,perc_total_infected_upper,cases
15945,2020-02-12,WA,0,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000009,0.000006,0.000013,67.4908,44.9939,101.236,0.000009,0.000006,0.000013,67.4908
15946,2020-02-13,WA,0,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000018,0.000012,0.000028,140.2360,93.4908,210.354,0.000018,0.000012,0.000028,72.7452
15947,2020-02-14,WA,0,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000030,0.000020,0.000045,230.7090,153.8060,346.064,0.000030,0.000020,0.000045,90.4730
15948,2020-02-15,WA,0,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000045,0.000030,0.000067,340.7070,227.1380,511.061,0.000045,0.000030,0.000067,109.9980
15949,2020-02-16,WA,0,0,0,NaN,NaN,NaN,NaN,NaN,...,0.000063,0.000042,0.000095,480.5200,320.3460,720.779,0.000063,0.000042,0.000095,139.8130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16261,2020-12-24,WA,230202,3162,3620650,1202.0,2276.71,17.14290,26822.6,0.084880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16262,2020-12-25,WA,233093,3184,3649210,1200.0,2269.71,9.57143,30902.6,0.073447,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16263,2020-12-26,WA,233093,3184,3649210,1200.0,1832.14,11.42860,24368.9,0.075184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16264,2020-12-27,WA,236719,3184,3710779,1200.0,2017.00,11.42860,29600.3,0.068141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
def compare_2():
    df_mine = get_df_adj()
    df_his = get_standard()
    p = figure(x_axis_type = 'datetime')
    #p.line(x = df_mine['date'], y = df_mine['infections'], legend_label = 'mine')
    p.line(x = df_his['date'], y = df_his['cases'], legend_label = 'his', color = 'green')
    p.line(x = df_mine['three_weeks'], y = df_mine['infections_by_ifr'].rolling(7).mean(), color = 'black',
          legend_label = 'ifr')
    return p

    
show(compare_2())


In [105]:
def sd_pos_rate():
    df = pd.read_csv('/home/henry/Downloads/results-20210102-204923.csv')
    df['date'] = pd.to_datetime(df['date'])
    p = figure(x_axis_type = 'datetime')
    p.line(x = df['date'], y = df['p'].rolling(7).mean())
    return p
    return df

show(sd_pos_rate())


In [131]:
def my_adj(window = 7):
    df = pd.read_csv('/home/henry/Downloads/results-20210102-214006.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df.assign(days_since_feb_12 = (df['date'] - datetime.datetime(2020, 2, 12)))
    days = [x.days for x in df['days_since_feb_12'].tolist() ]
    df = df.assign(days_since_feb_12 = days)
    df = df.assign(n_tests_7 = df['n_tests'].rolling(window).mean())
    df = df.assign(n_pos_7 = df['positive_increase'].rolling(window).mean())
    df = df.assign(pos_rate_7 = df['n_pos_7']/df['n_tests_7'])
    df = df.assign(prev = 1500/(50 + df['days_since_feb_12'] * np.sqrt(df['pos_rate_7'])))
    df = df[df['state'] == 'Washington']

    return df

my_adj()

    


/home/henry/Envs2/covid19/lib/python3.6/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


,state,positive_increase,date,n_tests,positive_rate,cases,deaths,population,days_since_feb_12,n_tests_7,n_pos_7,pos_rate_7,prev
11388,Washington,0,2020-01-22,0,NaN,0,0,7614893,-21,1.666701e+06,891.714286,0.000535,30.294303
11389,Washington,0,2020-01-23,0,NaN,0,0,7614893,-20,1.391857e+06,805.428571,0.000579,30.291472
11390,Washington,0,2020-01-24,0,NaN,0,0,7614893,-19,1.115751e+06,699.428571,0.000627,30.288167
11391,Washington,0,2020-01-25,0,NaN,0,0,7614893,-18,8.387396e+05,600.714286,0.000716,30.291843
11392,Washington,0,2020-01-26,0,NaN,0,0,7614893,-17,5.606989e+05,427.000000,0.000762,30.284147
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11728,Washington,3626,2020-12-27,0,NaN,273,0,7614893,319,0.000000e+00,2017.000000,inf,0.000000
11729,Washington,0,2020-12-28,0,NaN,2511,17,7614893,320,0.000000e+00,2017.000000,inf,0.000000
11730,Washington,1953,2020-12-29,0,NaN,2145,133,7614893,321,0.000000e+00,1719.571429,inf,0.000000
11731,Washington,2174,2020-12-30,0,NaN,1763,53,7614893,322,0.000000e+00,1851.285714,inf,0.000000


In [157]:
def get_ifr_rate(window = 7, plot_height = 350, plot_width = 350):
    df = pd.read_csv('/home/henry/Downloads/results-20210103-005919.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df[df['date'] >= datetime.datetime(2020,2,12)]
    df = df.assign(ifr = df['deaths']/df['infections'])
    df = df.dropna()
    dates = df['date'].tolist()

    days_since_feb_12 = []
    for i in dates:
        diff = i - datetime.datetime(2020,2, 12)
        days_since_feb_12.append((i, diff.days))
    df = df.assign(days_since_feb_12 = [x[1] for x in days_since_feb_12])

    x = df['days_since_feb_12'].tolist()
    X = list(zip(*[x]))
    y = df['ifr'].tolist()
    xm = sm.add_constant(X)
    model = sm.OLS(y, xm) 
    result = model.fit()
    y_hat = result.predict(xm)
    print(result.params[1], result.params[0])
    p3 = figure(plot_height = plot_height, plot_width = plot_width, title = 'IFR')
    p3.circle(x= x, y = y)
    p3.line(x = x, y = y_hat)
    grid = gridplot([p3], ncols = 2)
    return grid
show(get_ifr_rate(window = 7))


-7.968165601062178e-06 0.006927503857973091
